<a href="https://colab.research.google.com/github/helincesxyz/finetuneLLM/blob/main/ConversationalChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Install Dependencies



To enable CUDA:
First, you need to ensure that your Colab notebook is set to use a GPU. Here's how to enable a GPU in Colab:

1. Go to the "Edit" menu and select "Notebook settings" or "Runtime" > "Change runtime type".
2. Choose "GPU" and T4 as the hardware accelerator.
3. Click "Save" to apply the changes.
You can also use Colab Pro and choose V100 and High RAM for faster use. Choose A100 if youre gpu rich :)

Please Note:
- Using isolated sandbox systems such as Google Colabs significantly lowers the possibility of conflicts or problems with permissions with the system package manager.

- Generally, without taking certain unusual procedures, **you are not able to construct classic virtual environments in Colab using tools like virtualenv or venv**. It is therefore often safe to proceed with pip installations directly in Colab notebooks
 *(Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv)*

- The -U flag ensures that if the library is already installed, it will be upgraded to the specified version to prevent any bugs.

In [ ]:
%%bash
pip install -Uqqq pip
pip install -U bitsandbytes #supports cuda 12.2
pip install -U torch
pip install -qqq -U git+https://github.com/huggingface/transformers.git
pip install -qqq -U git+https://github.com/huggingface/peft.git
pip install -qqq -U git+https://github.com/huggingface/accelerate.git
pip install -U datasets
pip install -U loralib
pip install -q -U einops
pip install -q -U safetensors
pip install -q -U torch
pip install -q -U xformers

  Using cached torch-2.3.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached triton-2.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
Using cached torch-2.3.0-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
Using cached triton-2.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168.1 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2
    Uninstalling torch-2.2.2:
      Successfully uninsta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.14 requires torch<2.3,>=1.10, but you have torch 2.3.0 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
xformers 0.0.25.post1 requires torch==2.2.2, but you have torch 2.3.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchvis

In [ ]:
!nvidia-smi #check cuda

Sun Apr 28 13:15:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#1.1 Import Installed Libraries

1.
check to see if libraries are installed in the right place


In [ ]:
#check cuda lib
import os

# Check if the CUDA libraries directory exists
cuda_lib_dir = '/usr/local/cuda/lib64'
if os.path.exists(cuda_lib_dir):
    print("CUDA libraries directory found:", cuda_lib_dir)
else:
    print("CUDA libraries directory not found.")


CUDA libraries directory found: /usr/local/cuda/lib64


In [ ]:
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64"


In [ ]:
import bitsandbytes as bnb

In [ ]:
import os
import json
import pandas as pd
import torch
import torch.nn as nn
import transformers
from pprint import pprint
from datasets import load_dataset
from accelerate import Accelerator
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#1.2 Access HuggingFace API Token

In [ ]:
notebook_login()

#2. Upload Falcon 7B Model and QLora for FineTuning

We're going to import **Accelerate**: At Hugging Face, we created the 🤗 Accelerate library to help users easily train a 🤗 Transformers model on any type of distributed setup, whether it is multiple GPU's on one machine or multiple GPU's across several machines.

In [ ]:
from accelerate import Accelerator


https://vilsonrodrigues.medium.com/run-your-private-llm-falcon-7b-instruct-with-less-than-6gb-of-gpu-using-4-bit-quantization-ff1d4ffbabcc maybe try this too? #source

https://github.com/rohan-paul/LLM-FineTuning-Large-Language-Models/blob/main/Falcon-7B_FineTuning_with_PEFT_and_QLORA.ipynb #source

To upload our pre-trained model, you'll need WRITE access token from HuggingFace. You can also try more specialized models dependent on you liking.

In [ ]:
MODEL_NAME = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

https://huggingface.co/blog/4bit-transformers-bitsandbytes?source=post_page-----ff1d4ffbabcc--------------------------------

bitsandbytes article, set parameters according to this

In [ ]:
#analyze the current model's trainable parameters and total parameters
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

  # from pytroch

In [ ]:

model = prepare_model_for_kbit_training(model)
#IGNORE: You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


note:  QLoRA: 16 BIT works best with the hf bitsandbytes


In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


#3. Deployment Tests

#4. LOG PRIVATE DATASET

In [ ]:
prompt = """
<human>: Hello, how are you?
<assistant>:
""".strip()



In [ ]:

#CONFIGURE TEMPERATURE and topp
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 1
generation_config.top_p = 1
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

https://github.com/curiousily/Get-Things-Done-with-Prompt-Engineering-and-LangChain/blob/master/07.falcon-qlora-fine-tuning.ipynb #source

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )


print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Hello, how are you?
<assistant>: Hi! I'm doing great. How about you?
User 
CPU times: user 5.91 s, sys: 724 ms, total: 6.63 s
Wall time: 11 s


In [ ]:
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import pickle
import collections

import re
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

test data colab here: https://colab.research.google.com/drive/1_1hYJS2YMdJpTzgEIcqVvfaYX0YTIS2-#scrollTo=nK_1i4mql4kb

In [ ]:
import os
import json

PATH = "/content/coqa-dev-v1.0.json"

with open(PATH) as json_file:
    data = json.load(json_file)

In [ ]:
%%bash
head -c 1000 "coqa-dev-v1.0.json"

{
  "version": "1.0",
  "data": [
    {
      "source": "mctest",
      "id": "3dr23u6we5exclen4th8uq9rb42tel",
      "filename": "mc160.test.41",
      "story": "Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. \n\n\"What are you doing, Cotton?!\" \n\n\"I only wanted to

In [ ]:
data


In [ ]:
import json

def format_data_into_qa_pairs(data):
    qa_pairs = []


    for story in data["data"]:
        story_text = story["story"]


        for i in range(len(story["questions"])):
            question = story["questions"][i]["input_text"]
            answer = story["answers"][i]["input_text"]


            qa_pairs.append({"question": question, "answer": answer})

    return qa_pairs


with open("coqa-dev-v1.0.json", "r") as file:
    data = json.load(file)


qa_pairs = format_data_into_qa_pairs(data)


for i in range(5):
    print(qa_pairs[i])


{'question': 'What color was Cotton?', 'answer': 'white'}
{'question': 'Where did she live?', 'answer': 'in a barn'}
{'question': 'Did she live alone?', 'answer': 'no'}
{'question': 'Who did she live with?', 'answer': 'with her mommy and 5 sisters'}
{'question': 'What color were her sisters?', 'answer': 'orange and white'}


In [ ]:
qa_pairs

In [ ]:
qa_pairs[0]

{'question': 'Where was he born?', 'answer': 'China'}

In [ ]:
data_point = {
    "User": "Hello",
    "Prompt": "How can I assist you today?"
}


In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["question"]}
<assistant>: {data_point["answer"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data_point

{'User': 'Hello', 'Prompt': 'How can I assist you today?'}

In [ ]:
# Iterate over qa_pairs list and apply generate_and_tokenize_prompt function
processed_data = [generate_and_tokenize_prompt(qa_pair) for qa_pair in qa_pairs]


In [ ]:
import random

# Shuffle the qa_pairs list
random.shuffle(qa_pairs)

# Apply generate_and_tokenize_prompt function to each element of the shuffled list
data = list(map(generate_and_tokenize_prompt, qa_pairs))


In [ ]:
#data = processed_data["answers"].shuffle().map(generate_and_tokenize_prompt)

In [ ]:
data

In [ ]:
import torch
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, qa_pairs):
        self.qa_pairs = qa_pairs

    def __len__(self):
        return len(self.qa_pairs)

    def __getitem__(self, idx):
        return self.qa_pairs[idx]

# Convert qa_pairs to a PyTorch Dataset
dataset = QADataset(qa_pairs)

# Use the dataset in the Trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


#5. Fine-tuning The Model

In [ ]:
requires_grad=True

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


RuntimeError: Inference tensors cannot be saved for backward. To work around you can make a clone to get a normal tensor and use it in autograd.

#6. Save And Deploy The Model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "helincesxyz/convChatbot"

model.push_to_hub(
    PEFT_MODEL, token=True
)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

#6.1 Running The model

NEW PAPER FOR BETTER FINTEUNING:

In [ ]:

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: how have you been feeling lately?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


IndexError: tuple index out of range

#6.2 Deployment with Streamlit or FastAPI

In [ ]:
#not yet configured